# Langchain agent v1

In [ ]:
# Tools

from langchain.agents import initialize_agent, Tool, tool
from langchain.llms import OpenAI, OpenAIChat
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


In [ ]:
# initialize the LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0)
#llm = OpenAI(model_name="gpt-4", temperature=0)

In [ ]:
@tool("Rule_Violation")
def rule_violation_tool(context: str) -> str:
    """
    Identifies which (if any) rules in the context is violated by the last utterance in the context.
    """
    
    prompt = PromptTemplate(
    input_variables=["context"],
    template="""
    Below is a conversation context comprising a set of forum rules and a conversation thread defined by a set of utterances by different users. 
    \n\n CONVERSATION CONTEXT: \n{context}?")
    \n\n
    Identify which (if any) rules in the conversation context is violated by the last utterance when understood in context of the rest of the conversation thread. Also explain why/how it is a violation
    """)
    chain = LLMChain(llm=llm, prompt=prompt)
    
    result = chain.run(context)
    
    return result

@tool("Utterance_Selector")
def utterance_selector_tool(context: str) -> str:
    """
    Identifies the last utterance in the conversation context
    """
    prompt = PromptTemplate(
    input_variables=["context"],
    template="""
    Below is a conversation context comprising a set of forum rules and a conversation thread defined by a set of utterances by different users. 
    \n\n CONVERSATION CONTEXT: \n{context}?")
    \n\n
    Identify the last utterance in the conversation context
    """)
    chain = LLMChain(llm=llm, prompt=prompt)
    
    result = chain.run(context)
    
    return result


@tool("Rule_Exemplification")
def rule_exemplification_tool(context: str) -> str:
    """
    Identifies which (if any) rules in the context is exemplified by the last utterance in the context.
    """
    
    prompt = PromptTemplate(
    input_variables=["context"],
    template="""
    Below is a conversation context comprising a set of forum rules and a conversation thread defined by a set of utterances by different users. 
    \n\n CONVERSATION CONTEXT: \n{context}?")
    \n\n
    Identify which (if any) rules in the conversation context is exemplified (in a positive prosocial way) by the last utterance when understood in context of the rest of the conversation thread. Also explain why/how it exemplifies
    """)
    chain = LLMChain(llm=llm, prompt=prompt)
    
    result = chain.run(context)
    
    return result

@tool("Rule_Checker")
def rule_checker_tool(context: str) -> str:
    """
    Checks if the selected last utterance in the conversation context violates (in a bad way) or exemplifies (in a good way) the forum rules. 
    """
    
    prompt = PromptTemplate(
    input_variables=["context"],
    template="""
    Below is a conversation context comprising a set of forum rules and a conversation thread defined by a set of utterances (oldest first, newest last) by different users. 
    \n\n CONVERSATION CONTEXT: \n{context}?")
    \n\n
    Identify which (if any) rules in the utterance is exemplified (in a positive prosocial way) or violated (in a negative antisocial way) by the selected utterance when understood in context of the rest of the conversation thread. Also explain why/how it exemplifies or violates
    """)
    chain = LLMChain(llm=llm, prompt=prompt)
    
    result = chain.run(context)
    
    return result


@tool("Responder")
def responder(context: str) -> str:
    """
    Generates a response to a selected utterance if and when it has either violated or exemplified a forum rule.
    """
    print("Responder Input: ", context )
    prompt = PromptTemplate(
    input_variables=['context'],
    template="""
    Below is a conversation context comprising a set of forum rules and a conversation thread defined by a set of utterances by different users. 
    \n\n CONVERSATION CONTEXT: \n{context}?")
    \n\n
    Generate a response to a selected utterance either rewarding the user for a rule exemplifying utterance or warning the user of a forum violation whichever is applicable to the utterance.
    Also, your response should match the linguistic style of the selected utterance
    """)
    chain = LLMChain(llm=llm, prompt=prompt)
    
    result = chain.run(context)
    
    return result

In [ ]:
tools = [utterance_selector_tool, rule_checker_tool, responder]

In [ ]:
# Let's set up a custom Agent
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor



In [ ]:
prefix = """You are an online chatroom participant who promotes prosocial behavior by responding to comments. 
Respond ONLY to the last comment in the conversation thread as best you can. Respond only AFTER you have finished analyzing the last comment in the conversation. 

You have access to the following tools for analyzing the last comment:"""

format_instructions = """Use the following format:

Context: the input conversational context containing all the comments including the last comment you must generate a response to
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action, should also include the entire input context
Observation: the result of the action
... (this above Thought/Action/Action Input/Observation can repeat N times)
Thought: I now have a response. 
Final Answer: the final response to the input conversation context
"""

suffix = """
Begin! Again, remember to analyze the very last comment in the input context first, before responding to it. And respond ONLY to the last comment, not any others. 


Context: {input}
Thought:{agent_scratchpad}
"""


prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    format_instructions=format_instructions,
    input_variables=["input", "agent_scratchpad"]
)

In [ ]:
print(prompt.template)

In [ ]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [1]:
# EXAMPLE DATA

thread = """
USER1: So glad that we have somewhere to go. My mother needs a doctor.
USER2: Agreed. I need to bring my son who has injured his leg, if you don’t mind helping us
USER5: We need to help each other. Maybe G1 you could coordinate with G2?
USER3: I am not going to a mosque. Those people kill animals and eat cows there
USER4: These dogs have been flooding our villages
USER5: During a crisis like now, God says we should help people
USER3: Fuck the fucking muslims
USER5: Stop arguing. We need to help each other.
"""

thread5 = """
USER1: So glad that we have somewhere to go. My mother needs a doctor.
USER2: Agreed. I need to bring my son who has injured his leg, if you don’t mind helping us
USER5: We need to help each other. Maybe G1 you could coordinate with G2?
"""

thread3 = """
USER1: So glad that we have somewhere to go. My mother needs a doctor.
USER2: Agreed. I need to bring my son who has injured his leg, if you don’t mind helping us
USER5: We need to help each other. Maybe G1 you could coordinate with G2?
USER3: I am not going to a mosque. Those people kill animals and eat cows there
"""

thread4 = """
USER1: So glad that we have somewhere to go. My mother needs a doctor.
USER2: Agreed. I need to bring my son who has injured his leg, if you don’t mind helping us
USER5: We need to help each other. Maybe G1 you could coordinate with G2?
USER3: I am not going to a mosque. Those people kill animals and eat cows there
USER4: These dogs have been flooding our villages
"""
thread5R = """
USER1: So glad that we have somewhere to go. My mother needs a doctor.
USER2: Agreed. I need to bring my son who has injured his leg, if you don’t mind helping us
USER5: We need to help each other. Maybe G1 you could coordinate with G2?
USER3: I am not going to a mosque. Those people kill animals and eat cows there
USER4: These dogs have been flooding our villages
USER5: During a crisis like now, God says we should help people
"""

thread3R = """
USER1: So glad that we have somewhere to go. My mother needs a doctor.
USER2: Agreed. I need to bring my son who has injured his leg, if you don’t mind helping us
USER5: We need to help each other. Maybe G1 you could coordinate with G2?
USER3: I am not going to a mosque. Those people kill animals and eat cows there
USER4: These dogs have been flooding our villages
USER5: During a crisis like now, God says we should help people
USER3: Fuck the fucking muslims
"""


rules = """
1. Be friendly, respectful and polite
2. Do not use profanity or sexual language and/or images
3. No displays of hatred
4. Do not post violent content or incite violence
5. No political propaganda
6. No unfounded accusations or defamation of others
7. Follow grammatical rules, correct writing and language
8. Cite sources for factual information
9. Criticisms should be constructive
10. Stay on topic
"""


threads = [thread5, thread3, thread4 , thread5R, thread3R]

context5 = f"Rules:\n {rules} \n\n Conversation Thread: {thread5}"
context3 = f"Rules:\n {rules} \n\n Conversation Thread: {thread3}"
context4 = f"Rules:\n {rules} \n\n Conversation Thread: {thread4}"
context5R = f"Rules:\n {rules} \n\n Conversation Thread: {thread5R}"
context3R = f"Rules:\n {rules} \n\n Conversation Thread: {thread3R}"

contexts = [context5, context3, context4, context5R, context3R]

In [ ]:
agent_executor.run(context3)

# Langchain Agent v2

Date: 4/4/2023

Doing a simple chain here (not Agent and tools)

In [9]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain


#llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)
llm = OpenAI(temperature=0.0)

In [10]:
template_id_last_utterance = """
    Below is a conversation thread defined by a set of utterances by different users. Identify the last utterance in the thread.
    \n\n 
    Conversation Thread: {thread}\n
    Last Utterance: 
    """
prompt_id_last_utterance = PromptTemplate(
    input_variables=["thread"],
    template=template_id_last_utterance,
)

chain_id_last_utterance = LLMChain(llm=llm, prompt=prompt_id_last_utterance)

In [11]:
template_utterance_expander = """
Below is a conversation context comprising a conversation thread and a selected utterance.
Elaborate on what the selected utterance actually means in the context of what was already stated in the conversation thread.
Specifically, elaborate on all the referring expressions and what they could mean. 
Focus on explicating if words have negative or positive connotations in this context, and what/who group (mentioned previously in context) they refer to.
Remember, words might specific (even unusual) meaning in this particular context, different from other contexts. For example animals and inanimate objects may actually refer to groups or types of people.

\n\n CONVERSATION CONTEXT: \n
    Thread:\n{thread}\n
    Selected Utterance: \n{utterance}\n
    Deeper Meaning of selected utterance:
"""

prompt_utterance_expander = PromptTemplate(
    input_variables=["thread","utterance"],
    template=template_utterance_expander,
)

chain_utterance_expander = LLMChain(llm=llm, prompt=prompt_utterance_expander)


In [12]:
template_rule_checker="""
 Below is a conversation context comprising a set of forum rules, a conversation thread, a selected utterance and its deeper meaning. 
 Identify which (if any) rules in the selected utterance is exemplified (in a positive prosocial way) or violated (in a negative antisocial way) by the selected utterance when understood in context of the rest of the conversation thread. Also explain why/how it exemplifies or violates

    \n\n CONVERSATION CONTEXT: \n
    Rules: \n{rules}\n
    Conversation thread: \n{thread}\n
    Selected Utterance: {utterance}\n
    Deeper meaning: {meaning}\n
    Rule exemplifcation/violation: 
"""

prompt_rule_checker = PromptTemplate(
    input_variables=["rules","thread","utterance", "meaning"],
    template=template_rule_checker,
)

chain_rule_checker = LLMChain(llm=llm, prompt=prompt_rule_checker)

In [13]:
template_dsr="""
Below is a conversation context comprising a conversation thread, a selected utterance and its deeper meaning.
Identify if the user (author) of the selected utterance is targeting "social regard" towards a particular group of people.
The "social regard" could be positive (like empathy, respect, helpfulness) or negative (like contempt, dehumanization, harming).
Identify the specific group being targeted as well as the type of social regard directed towards the group.

\n\n CONVERSATION CONTEXT: \n
    Thread:\n{thread}\n
    Selected Utterance: \n{utterance}\n
    Deeper Meaning: \n{meaning}\n
    Directed Social Regard toward a particular Group of people:
"""

prompt_dsr = PromptTemplate(
    input_variables=["thread","utterance", "meaning"],
    template=template_dsr,
)

chain_dsr = LLMChain(llm=llm, prompt=prompt_dsr)

In [14]:
template_respond="""
 Below is a conversation context comprising a set of forum rules, a conversation thread, a selected utterance, its deeper meaning, and an analysis of whether the selected utterance violated or exemplified any rules. 
 Generate a response to a selected utterance either rewarding the user for a rule exemplifying utterance or warning the user of a forum violation whichever is applicable to the utterance.
 When the user of the selected utterance is displaying negative social regard towards a group, your response must remind the user of what qualities, desires, beliefs, intentions, morals or values they share in common with the targeted group. 
 Your response should address the user of the selected utterance, match the linguistic style of the thread, and must display prosocial qualities of empathy, compassion, and curiosity.
 Keep your response short (under 50 words)
    
    \n\n CONVERSATION CONTEXT: \n
    Rules:\n{rules}\n
    Thread:\n{thread}\n
    Selected Utterance: \n{utterance}\n
    Deeper Meaning: \n{meaning}\n
    Directed Social regard towards a group: \n{dsr}\n
    Analysis: \n{analysis}\n
    Response:
"""

prompt_respond = PromptTemplate(
    input_variables=["thread","utterance", "analysis", "rules", "dsr", "meaning"],
    template=template_respond,
)

chain_respond = LLMChain(llm=llm, prompt=prompt_respond)

In [16]:
thread = thread4
utterance = chain_id_last_utterance.run(thread=thread)
meaning = chain_utterance_expander.run(thread=thread, utterance=utterance)
analysis = chain_rule_checker.run(rules=rules, thread=thread, utterance=utterance, meaning=meaning)
dsr = chain_dsr.run(thread=thread, utterance=utterance, meaning=meaning)
response = chain_respond.run(rules=rules, thread=thread, utterance=utterance, analysis=analysis, dsr=dsr, meaning=meaning)
print("\n",response)


 
USER4, we are all part of the same community and share the same desire for safety and security. Let's focus on finding solutions that benefit everyone.
